# Accelerating numpy with Pythran

Pythran might be a way to accelerate a bottleneck function in your code by compiling it to C++.

We begin with some imports including the `pythran` magic extension. This magic allows you to use pythran like a just-in-time compiler in ipython and jupyter

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import pythran

import altair as alt

from IPython.display import Markdown, display

%load_ext pythran.magic

Create a print function that lets us use markdown formatting

In [2]:
def printmd(string):
    display(Markdown(string))

# Long array operation

In [25]:
def getATPNumpy(T,P):
    A00 = 1
    A_t_p = ((A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)) + (A00*(T**4)))    +
             (A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)) + (A00*(T**4)))*P + 
             (A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)))*(P**2) + 
             (A00 + (A00*T) + (A00*(T**2)))*(P**3))
    return A_t_p

### Compile the pythran function using the cell magic

In [26]:
%%pythran 
#pythran export getATPPythran(float64[:,:] order(C), float64[:,:] order(C))
def getATPPythran(T,P):
    A00 = 1
    A_t_p = ((A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)) + (A00*(T**4)))    +
             (A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)) + (A00*(T**4)))*P + 
             (A00 + (A00*T) + (A00*(T**2)) + (A00*(T**3)))*(P**2) + 
             (A00 + (A00*T) + (A00*(T**2)))*(P**3))
    return A_t_p

### Generate the array

In [46]:
A00 = 1.0
np.random.seed(3)
def generateArray(z:np.ndarray,N=10):
    T = np.random.standard_normal((len(z),N))
    return T

In [47]:
T = generateArray(z=z,N=1000)

In [29]:
np.testing.assert_array_equal(getATPPythran(T=T,P=-z[:,np.newaxis]),getATPNumpy(T=T,P=-z[:,np.newaxis]))

AssertionError: 
Arrays are not equal

Mismatched elements: 343 / 199000 (0.172%)
Max absolute difference: 2.38418579e-07
Max relative difference: 3.91785661e-16
 x: array([[3.548403e+02, 6.630850e+01, 4.424440e+01, ..., 5.934262e+01,
        2.727742e+02, 2.860142e+01],
       [2.682958e+02, 9.579535e+02, 8.703086e+02, ..., 1.998691e+02,...
 y: array([[3.548403e+02, 6.630850e+01, 4.424440e+01, ..., 5.934262e+01,
        2.727742e+02, 2.860142e+01],
       [2.682958e+02, 9.579535e+02, 8.703086e+02, ..., 1.998691e+02,...

In [50]:
T = generateArray(z=z,N=100000)

In [51]:
printmd("**Numpy**")
%timeit -n 1 -r 1 getATPNumpy(T=T,P=-z[:,np.newaxis])
printmd("**Pythran**")
%timeit -n 1 -r 1 getATPPythran(T=T,P=-z[:,np.newaxis])

**Numpy**

4.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Pythran**

375 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
